In [157]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
import xml.etree.ElementTree as et
from io import BytesIO
from typing import List, Dict, Optional
from matplotlib import pyplot as plt
import numpy as np

# Классы и методы

In [3]:
class FedStatIndicator:
    def __init__(self, indicator_id):
        self.id = indicator_id
        self._filters_raw = None
        self._raw_data = None

    def _get_filters_raw(self):
        """Получает сырые данные фильтров"""
        html = requests.get(f'https://www.fedstat.ru/indicator/{self.id}')
        if html.status_code == 200:
            soup = BeautifulSoup(html.text, "lxml")
            script = soup.find_all("script")[11].text
            pattern = r"filters:\s*(\{.*?\})(?=\s*,\s*left_columns)"
            match = re.search(pattern, script, re.DOTALL).group(0)
            filters = "{" + match + "}"
            filters = re.sub(r'([{,]\s*)(\w+)(\s*:)', r'\1"\2"\3', filters)
            filters = filters.replace("'", '"')
            self._filters_raw = json.loads(filters)['filters']
            return self._filters_raw
        else:
            raise requests.RequestException(f"Ошибка HTTP-запроса")

    def get_filters_text(self):
        """
        Возвращает код и название показателя, доступного для фильтрации
        """
        filters = self._get_filters_raw()
        filter_values = {key : filters[key]['title'] for key in list(filters.keys())[1:]}
        return filter_values

    def get_filter_values(self):
        """
        Возвращает названия и коды доступных значений для выбранных фильтров
        """
        filters = self._get_filters_raw()
        filter_codes = self.get_filters_text()
        
        categories =[]
        for key in filter_codes.keys():
            categories.append({
                key: list(filters[key]['values'].keys())
            })
        filter_ids = [f"{k}_{val}" for item in categories for k, v in item.items() for val in v]
        return filter_ids

    def load_raw_indicator(self, data_type: str = "excel", filter_ids: List["str"] =  None):
        """
       Загружает данные индикатора через API

       :param data_type: тип возвращаемых данных
       :param filter_ids: список с кодами выбранных показателей
       :return: DataFrame с данными
        """

        if self._raw_data is not None:
            return self._raw_data

        if filter_ids is None:
            filter_ids = self.get_filter_values()

        data = {
                "lineObjectIds": ["0",  "57831", "58335", "58274", "30611"],
                "columnObjectIds": ["3", "33560"],
                "selectedFilterIds": [
                filter_ids
                ]
            }
        params = {
            "format" : data_type,
            "id" : self.id  
        }
        try: 
            response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
            response.raise_for_status()
            if "excel" in response.headers.get("Content-Type"):
                self._raw_data = pd.read_excel(BytesIO(response.content), engine = "xlrd", header = 2) 
                return self._raw_data
            else:
                raise ValueError(
                    f"Ответ не содержит {data_type} файла"
                )
        except requests.RequestException as e:
            raise requests.RequestException(f"Ошибка HTTP-запроса: {str(e)}")

    @staticmethod
    def _get_min_age(age_str):
        numbers = re.findall(r'\d+', age_str)
        return int(numbers[0]) if numbers else None

    @staticmethod
    def _get_max_age(age_str):
        numbers = re.findall(r"\d+", age_str)
        return int(numbers[-1]) if numbers else None

    @staticmethod   
    def _categorize_age(age_str):
        numbers = re.findall(r'\d+', age_str)
        words = re.findall(r'\b[а-яА-ЯёЁ]+\b', age_str, flags=re.IGNORECASE)
        if len(numbers) == 1 and len(words) <=3:
            return 1
        elif len(numbers) == 2 and len(words) <= 1:
            if int(numbers[-1]) - int(numbers[0]) == 4:
                return 2
            else:
                return 3
        else:
            return 4
    
    def _preprocess_dataframe(self, df):
     
        df.drop([df.columns[0], df.columns[4]], axis = 1, inplace = True)
        df = df.drop(0, axis = 0).reset_index(drop = True)

        df.rename(columns = {
            'Unnamed: 1' : "region",
            "Unnamed: 2" : "age",
            "Unnamed: 3" : "settlement"
            }, inplace = True)

        df.region = df.region.str.strip()
        years = [col for col in df.columns if col.isdigit()]
        df[years] = df[years].astype("Int64")
        
        return df

    def _remove_districts(self, df):

        """
        Удаляет федеральные округа из данных.
        :param df: DataFrame, предварительно обработанный
        :return: DataFrame без округов
        """
        
        rm_districts = '|'.join([
            r'Южный федеральный',
            r'Сибирский федеральный',
            r'Северо-Кавказский федеральный',
            r'Дальневосточный федеральный'
        ])

        districts_mask = df['region'].str.contains(rm_districts, case=False)
        df = df[~districts_mask].copy().reset_index(drop = True)
        return df

    def _change_districts(self, df):
        
        """
        Группирует регионы по федеральным округам.
        :param df: DataFrame после удаления округов
        :return: DataFrame с добавленными строками по округам
        """
        districts_config = {
            'Дальневосточный федеральный округ' : '|'.join([
                r'Бурятия',
                r'Забайкал',
                r'Саха',
                r'Якутия',
                r'Камчат',
                r'Приморск',
                r'Хабаровский',
                r'Амурская',
                r'Магаданская',
                r'Сахалинская',
                r'Еврейская автономная область',
                r'Чукотский автономный округ']),
            'Сибирский федеральный округ' : '|'.join([
                r'Алтай',
                r'Тыва',
                r'Хакасия',
                r'Красноярский',
                r'Иркутская',
                r'Кемеровская',
                r'Кузбасс',
                r'Новосибирская',
                r'Омская',
                r'Томская']),
            'Южный федеральный округ': '|'.join([
                r'Адыгея', 
                r'Калмыкия',
                r'Краснодарский',
                r'Астраханская',
                r'Волгоградская',
                r'Ростовская',
                r'Крым',
                r'Севастополь'
                ]),
            'Северо-Кавказский федеральный округ' :  '|'.join([
                r'Дагестан',
                r'Ингушетия',
                r'Кабардин',
                r'Балкар',
                r'Карачаев',
                r'Черкес',
                r'Осетия',
                r'Алания',
                r'Чечен',
                r'Ставрополь'
            ])   
        }
        aggregated_dfs = []
        for district_name, regions in districts_config.items():
            mask = df['region'].str.contains(regions, case = False, na = False)
            district_df = df[mask].copy()
            if not district_df.empty:
                agg_df = district_df.groupby(
                    ['age', 'settlement'],
                    as_index = False
                ).sum(numeric_only = True)
                agg_df.insert(0, 'region', district_name)
                aggregated_dfs.append(agg_df)
        df = pd.concat([df] + aggregated_dfs, ignore_index = True)
        return df   

    def _add_mid_year_values(self, df):
        
        if df is None:
            df = self._change_districts()
        
        df['min_age']  = df['age'].apply(self._get_min_age)
        df['max_age'] = df['age'].apply(self._get_max_age)
        df['age_category'] = df['age'].apply(self._categorize_age)

        

        min_year = min([int(col) for col in df.columns if col.isdigit()])
        df_mid = pd.DataFrame()

        for col in df.columns:
            if col.isdigit():
                if int(col) > min_year:
                    df_mid[f"{col}mid"] = (df[f"{col}"] + df[f"{int(col)-1}"])/2
                df_mid[f"{col}end"] = df[col]
            else:
                df_mid[col] = df[col]        
        
        df_mid = df_mid.groupby('region', sort = False).apply(
                lambda x: x.sort_values(['age_category', 'min_age', 'max_age'])
                ).reset_index(drop = True)
        return df_mid

    def get_processed_data(self, data_type: str = "excel", filter_ids: List["str"] =  None ):
        
        """
        Загружает, очищает и агрегирует данные Росстата для дальнейшего анализа.

        Последовательность обработки данных:
        1. Загружает исходные данные индикатора через API (метод `load_raw_indicator`).
        2. Выполняет предварительную очистку таблицы:
        - Переименовывает столбцы,
        - Удаляет лишние строки и столбцы,
        - Приводит числовые столбцы к типу Int64.
        3. Удаляет из данных строки с федеральными округами (оставляет только регионы).
        4. Агрегирует данные по федеральным округам:
        - Объединяет данные по регионам в укрупнённые федеральные округа.
        5. Добавляет дополнительные столбцы со значениями за середину года (среднее между двумя годами).

        :param data_type: Формат загружаемых данных с сайта (по умолчанию "excel").
        :param filter_ids: Список кодов фильтров для отбора данных (если не указан, загружаются все доступные).
        :return: pandas.DataFrame с итоговыми очищенными и агрегированными данными.
        """

        raw_df = self.load_raw_indicator(
            data_type = data_type,
            filter_ids = filter_ids
        )
        df = self._preprocess_dataframe(raw_df)
        df = self._remove_districts(df)
        df = self._change_districts(df)
        df = self._add_mid_year_values(df)
        return df

In [ ]:
#Load the data
def get_filters_raw(id):
    """
    Код принимает на вход id показателя и возвращает список с фильтрами
    Это вспомогательная функция
    """
    try: 
        html = requests.get(f'https://www.fedstat.ru/indicator/{id}')
        soup = BeautifulSoup(html.text, "lxml")
        script = soup.find_all("script")[11].text
        pattern = r"filters:\s*(\{.*?\})(?=\s*,\s*left_columns)"
        match = re.search(pattern, script, re.DOTALL).group(0)
        filters = "{" + match + "}"
        filters = re.sub(r'([{,]\s*)(\w+)(\s*:)', r'\1"\2"\3', filters)
        filters = filters.replace("'", '"')
        filters = json.loads(filters)['filters']
        return filters
    except requests.RequestException as e:
        raise requests.RequestException(f"Ошибка HTTP-запроса: {str(e)}")

def get_filters_text(filters):
    """
    Возвращает код и название показателя, доступного для фильтрации
    """
    filter_values = {key : filters[key]['title'] for key in list(filters.keys())[1:]}
    return filter_values

def get_values(filters):
    """
    Возвращает названия и коды доступных значений для выбранных фильтров
    """
    filter_codes = filters.keys()
    categories =[]
    for key in filter_codes:
        categories.append({
            key: list(filters[key]['values'].keys())
        })
    ids = [f"{k}_{val}" for item in categories for k, v in item.items() for val in v]
    return ids

def load_indicator(id, data_type, ids):
    """
    На вход передается код индикатора, тип данных, коды показателей для фильтрации
    Возвращает датафрейм с данными
    """
    data = {
            "lineObjectIds": ["0",  "57831", "58335", "58274", "30611"],
            "columnObjectIds": ["3", "33560"],
            "selectedFilterIds": [
                ids
            ]
        }
    params = {
        "format" : data_type,
        "id" : id
    }
    try: 
        response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
        if "excel" in response.headers.get("Content-Type"):
            return pd.read_excel(BytesIO(response.content), engine = "xlrd")
        else:
            raise ValueError(
                f"Ответ не содержит {data_type} файла"
            )
    except requests.RequestException as e:
        raise requests.RequestException(f"Ошибка HTTP-запроса: {str(e)}")  

# Женское население

In [4]:
women_population = FedStatIndicator(indicator_id = 33459)

In [5]:
df_women = women_population.get_processed_data()

C:\Users\User\AppData\Local\Temp\ipykernel_8912\556666468.py:236: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mid = df_mid.groupby('region', sort = False).apply(


In [7]:
df_women['age'] = df_women['age'].str.replace(r'\s*(лет|года|год)$', '', regex = True).str.strip()
df_women = df_women.drop_duplicates(subset = ['region', 'age', 'settlement'], keep = 'last')

In [91]:
df_women.to_excel("women_population.xlsx", engine = "openpyxl", index = False)

## SDMX data

In [ ]:
#All values
data = {
        "lineObjectIds": ["0", "30611", "58335", "57831", "58274"],
        "columnObjectIds": ["3", "33560"],
        "selectedFilterIds": ids
}
params = {
    "format" : "sdmx",
    "id" : "33459"
}
response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
print(response.status_code, response.headers)

In [7]:
# Adding  regions
regions = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="57831" for val in v]

In [8]:
#Adding years
years = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="3" for val in v]

In [9]:
#Adding settlement type
settlement_type = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="58274" for val in v]
settlement_type

['58274_1744150', '58274_1750789', '58274_1750788']

In [10]:
#Adding age categories
age =  [f"{k}_{val}" for item in categories for k, v in item.items() if k =="58335" for val in v]

In [12]:
#collect regional data
data = {
        "lineObjectIds": ["0", "30611", "58335", "57831", "58274"],
        "columnObjectIds": ["3", "33560"],
        "selectedFilterIds": [
            "0_33459",
            years,
            "30611_950458",
            "33560_1540248",
            regions,
            "58274_1744150",
            "58335_1709566"
        ]
    }
params = {
    "format" : "sdmx",
    "id" : "33459"
}
response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
print(response.status_code, response.headers)

200 {'Content-Type': 'text/xml', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Sat, 12 Jul 2025 10:39:48 GMT', 'Set-Cookie': 'JSESSIONID=20E85FCD89FE3297FE03E7FAF9608248; Path=/; Secure; HttpOnly, session-cookie=18517ad4dc4c6a7edcd5a505204218c4d2506ea5bd73d1c7df057e3ecf38e193bc79ec9cdd66ebf6efc4768835a5ee4b; Max-Age=86400; Path=/; secure; HttpOnly', 'Cache-Control': 'no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': '0', 'Pragma': 'public', 'Content-Disposition': "attachment; filename*=UTF-8''data.xml", 'vary': 'accept-encoding', 'Content-Language': 'ru', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'Content-Security-Policy': 'upgrade-insecure-requests', 'Referrer-Policy': 'strict-origin'}


In [13]:
# Save the data in a file
with open("regions.xml", "wb") as file:
    file.write(response.content)

In [41]:
#Transform sdmx file into pandas dataframe
def read_sdmx(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    namespaces = {
        "generic": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic",
        "message": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/message",
        "structure": 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/structure'
    } 

    dataset = root.find(".//message:DataSet", namespaces)
    data = []
    for series in dataset.findall(".//generic:Series", namespaces):
        observations = series.findall(".//generic:Obs", namespaces)
        pair = {}
        for obs in observations:
            time = obs.find(".//generic:Time", namespaces).text
            value = obs.find(".//generic:ObsValue", namespaces).attrib['value']
            region = obs.find(".//")
            data.append({
                "year" : time,
                "population" : value
                })
    return pd.DataFrame(data)

data = read_sdmx("regions.xml")
data

,year,population
0,1990,78549601
1,1991,78817695
2,1992,78914788
3,1993,78899303
4,1994,78770458
...,...,...
3502,1996,3833075
3503,1999,3700204
3504,2000,3620686
3505,2001,3620686


## Excel data

In [40]:
#Let's get the data in excel
def load_indicator(id, data_type, regions, years, settlement_type, age):
    data = {
            "lineObjectIds": ["0",  "57831", "58335", "58274", "30611"],
            "columnObjectIds": ["3", "33560"],
            "selectedFilterIds": [
                regions,
                "0_33459",
                years,
                "30611_950458",
                "33560_1540248",
                settlement_type,
                age
            ]
        }
    params = {
        "format" : data_type,
        "id" : id
    }
    response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
    return response
# print(response.status_code, response.headers)

In [7]:
#Load the data
df = pd.read_excel("all_data.xls", header = 2)
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,1990,1991,1992,1993,1994,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,NaN,NaN,NaN,NaN,NaN,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,...,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января
1,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,все население,человек,78549601,78817695,78914788,78899303,78770458,...,78958558,79137859,79207619,79202094,79219390,78952485,78601633,78354712,78233436,78154709
2,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,городское население,человек,57983496,58303864,58267844,57878461,57688561,...,59279662,59512332,59660317,59755068,59858643,59691444,59487917,59352665,59312967,59399259
3,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,сельское население,человек,20566105,20513831,20646944,21020842,21081897,...,19678896,19625527,19547302,19447026,19360747,19261041,19113716,19002047,18920469,18755450
4,Численность постоянного населения - женщин по ...,Российская Федерация,0,все население,человек,1035959,957048,868672,770771,675810,...,942238,910777,814676,771071,714593,688397,673494,632575,611254,593327


In [8]:
#Remove unnecassary columns
df.drop([df.columns[0], df.columns[4]], axis = 1, inplace = True)
df = df.drop(0, axis = 0).reset_index(drop = True)

In [9]:
# Rename
df.rename(columns = {
    'Unnamed: 1' : "region",
    "Unnamed: 2" : "age",
    "Unnamed: 3" : "settlement"
    }, inplace = True)

In [10]:
df.region = df.region.str.strip()

In [11]:
#Remove unnecessary districts
rm_districts = '|'.join([
    r'Южный федеральный',
    r'Сибирский федеральный',
    r'Северо-Кавказский федеральный',
    r'Дальневосточный федеральный'
])
districts_mask = df['region'].str.contains(rm_districts, case=False)
df= df[~districts_mask].reset_index(drop = True)

In [12]:
years = [col for col in df.columns if col.isdigit()]
df[years] = df[years].astype("Int64")

In [13]:
def get_min_age(age):
    numbers = re.findall(r'\d+', age)
    return int(numbers[0]) if numbers else None
    
def get_max_age(age):
    numbers = re.findall(r"\d+", age)
    return int(numbers[-1]) if numbers else None
    
def categorize_age(age_str):
    age_str = age_str.lower()
    numbers = re.findall(r'\d+', age_str)
    words = re.findall(r'\b[а-яА-ЯёЁ]+\b', age_str, flags=re.IGNORECASE)
    if "-" in age_str and len(words) == 1:
        age_diff = int(numbers[1]) - int(numbers[0])
        return 2 if age_diff == 4 else 3
    elif "всего" in age_str or not numbers or len(words) > 3:
        return 4
    else:
        return 1

df['min_age']  = df['age'].apply(get_min_age)
df['max_age'] = df['age'].apply(get_max_age)
df['age_category'] = df['age'].apply(categorize_age)

In [14]:
#Add mid-year value
min_year = min([int(col) for col in df.columns if col.isdigit()])
df_mid = pd.DataFrame()

for col in df.columns:
    if col.isdigit():
        if int(col) > min_year:
            df_mid[f"{col}mid"] = (df[f"{col}"] + df[f"{int(col)-1}"])/2
        df_mid[f"{col}end"] = df[col]
    else:
        df_mid[col] = df[col]
df_mid.head()

,region,age,settlement,1990end,1991mid,1991end,1992mid,1992end,1993mid,1993end,...,2022end,2023mid,2023end,2024mid,2024end,2025mid,2025end,min_age,max_age,age_category
0,Российская Федерация,Всего,все население,78549601,78683648.0,78817695,78866241.5,78914788,78907045.5,78899303,...,78601633,78478172.5,78354712,78294074.0,78233436,78194072.5,78154709,NaN,NaN,4
1,Российская Федерация,Всего,городское население,57983496,58143680.0,58303864,58285854.0,58267844,58073152.5,57878461,...,59487917,59420291.0,59352665,59332816.0,59312967,59356113.0,59399259,NaN,NaN,4
2,Российская Федерация,Всего,сельское население,20566105,20539968.0,20513831,20580387.5,20646944,20833893.0,21020842,...,19113716,19057881.5,19002047,18961258.0,18920469,18837959.5,18755450,NaN,NaN,4
3,Российская Федерация,0,все население,1035959,996503.5,957048,912860.0,868672,819721.5,770771,...,673494,653034.5,632575,621914.5,611254,602290.5,593327,0.0,0.0,1
4,Российская Федерация,0,городское население,731046,698696.5,666347,630439.5,594532,555098.0,515664,...,506226,490632.5,475039,466675.0,458311,453286.0,448261,0.0,0.0,1


In [15]:
#Fix federal districts

dfo_regions = [
    'Республика Бурятия',
    'Забайкальский край',
    'Республика Саха (Якутия)',
    'Камчатский край',
    'Приморский край',
    'Хабаровский край',
    'Амурская область',
    'Магаданская область',
    'Сахалинская область',
    'Еврейская автономная область',
    'Чукотский автономный округ'
]
dfo_mask = df_mid['region'].isin(dfo_regions)
dfo = df_mid[dfo_mask]
dfo_agg = dfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
dfo_agg.insert(0, 'region', 'Дальневосточный федеральный округ')

In [16]:
sfo_regions = ['Республика Алтай',
    'Республика Тыва',
    'Республика Хакасия',
    'Алтайский край',
    'Красноярский край',
    'Иркутская область',
    'Кемеровская область - Кузбасс',
    'Новосибирская область',
    'Омская область',
    'Томская область']
sfo_mask = df_mid['region'].isin(sfo_regions)
sfo = df_mid[sfo_mask]
sfo_agg = sfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
sfo_agg.insert(0, 'region', 'Сибирский федеральный округ')

In [17]:
ufo_pattern = '|'.join([
    r'Адыгея', 
    r'Калмыкия',
    r'Краснодарский',
    r'Астраханская',
    r'Волгоградская',
    r'Ростовская',
    r'Крым',
    r'Севастополь'
])

skfo_pattern = '|'.join([
    r'Дагестан',
    r'Ингушетия',
    r'Кабардин',
    r'Балкар',
    r'Карачаев',
    r'Черкес',
    r'Осетия',
    r'Алания',
    r'Чечен',
    r'Ставрополь'
])

ufo_mask = df_mid['region'].str.contains(ufo_pattern, case = False, na = False)
ufo = df_mid[ufo_mask]
ufo_agg = ufo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
ufo_agg.insert(0, 'region', 'Южный федеральный округ')

skfo_mask = df_mid['region'].str.contains(skfo_pattern, case = False, na = False)
skfo = df_mid[skfo_mask]
skfo_agg = skfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
skfo_agg.insert(0, 'region', 'Северо-Кавказский федеральный округ')

df_mid = pd.concat([df_mid, ufo_agg, skfo_agg, sfo_agg], ignore_index = True)


In [19]:
df_mid = df_mid.groupby('region', sort = False).apply(
    lambda x: x.sort_values(['age_category', 'min_age', 'max_age'])
).reset_index(drop = True)
df_mid.head()

C:\Users\User\AppData\Local\Temp\ipykernel_5108\2115352455.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mid = df_mid.groupby('region', sort = False).apply(


,region,age,settlement,1990end,1991mid,1991end,1992mid,1992end,1993mid,1993end,...,2022end,2023mid,2023end,2024mid,2024end,2025mid,2025end,min_age,max_age,age_category
0,Российская Федерация,0,все население,1035959,996503.5,957048,912860.0,868672,819721.5,770771,...,673494,653034.5,632575,621914.5,611254,602290.5,593327,0.0,0.0,1
1,Российская Федерация,0,городское население,731046,698696.5,666347,630439.5,594532,555098.0,515664,...,506226,490632.5,475039,466675.0,458311,453286.0,448261,0.0,0.0,1
2,Российская Федерация,0,сельское население,304913,297807.0,290701,282420.5,274140,264623.5,255107,...,167268,162402.0,157536,155239.5,152943,149004.5,145066,0.0,0.0,1
3,Российская Федерация,1 год,все население,1132245,1082223.5,1032202,996674.5,961147,915514.0,869881,...,689192,681807.5,674423,653844.5,633266,623035.5,612805,1.0,1.0,1
4,Российская Федерация,1 год,городское население,799394,763825.0,728256,697132.0,666008,627874.0,589740,...,519345,513412.5,507480,491841.0,476202,468292.5,460383,1.0,1.0,1


In [ ]:
#Highlight "total" rows with red color
def highlight_total(row, color = 'red'):
    return [f"background-color: {color}" for _ in row]
df_styled = df_mid.style.apply(
    lambda row : highlight_total(row) if row['age'] == 'Всего' else ['']*len(row),
    axis = 1
)
df_styled

In [ ]:
#Save in a csv file
df_mid.drop(columns = ['age_category', 'min_age', 'max_age'], inplace = True)
df_mid.to_excel("sorted_data.xlsx", index = False)
df_styled.to_excel("sorted_data_highlighted.xlsx", index = False)

# Мужское население

In [8]:
men_population = FedStatIndicator(indicator_id = 31548)

In [9]:
df_men = men_population.get_processed_data()

C:\Users\User\AppData\Local\Temp\ipykernel_8912\556666468.py:236: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mid = df_mid.groupby('region', sort = False).apply(


In [202]:
df_men.head()

,region,age,settlement,1990end,1991mid,1991end,1992mid,1992end,1993mid,1993end,...,2022end,2023mid,2023end,2024mid,2024end,2025mid,2025end,min_age,max_age,age_category
0,Российская Федерация,0,все население,1087571,1047535.0,1007499,959429.0,911359,862700.5,814042,...,714446,691336.0,668226,659125.5,650025,640359.5,630694,0.0,0.0,1
1,Российская Федерация,0,городское население,767215,735244.0,703273,664478.0,625683,585280.0,544877,...,537022,519595.0,502168,495496.5,488825,483194.5,477564,0.0,0.0,1
2,Российская Федерация,0,сельское население,320356,312291.0,304226,294951.0,285676,277420.5,269165,...,177424,171741.0,166058,163629.0,161200,157165.0,153130,0.0,0.0,1
3,Российская Федерация,1 год,все население,1178636,1130834.5,1083033,1046561.0,1010089,961233.5,912378,...,731314,723334.5,715355,692132.5,668910,660203.0,651496,1.0,1.0,1
4,Российская Федерация,1 год,городское население,833156,798796.5,764437,733271.5,702106,661209.0,620312,...,551322,544867.5,538413,520807.5,503202,497014.5,490827,1.0,1.0,1


In [10]:
df_men['age'] = df_men['age'].str.replace(r'\s*(лет|года|год)$', '', regex = True).str.strip()
df_men = df_men.drop_duplicates(subset = ['region', 'age', 'settlement'], keep = 'last')

In [90]:
df_men.to_excel("men_population.xlsx", index = False)

In [17]:
columns = ['region', 'age', 'settlement']
diff = df_men[columns].merge(df_women[columns], how='outer', indicator=True)

In [31]:
diff_values = diff[diff['_merge'] == 'left_only']

mask = df_men[['region', 'age', 'settlement']].apply(tuple, axis=1).isin(
    diff_values[['region', 'age', 'settlement']].apply(tuple, axis=1)
)
df_men = df_men[~mask]


In [34]:
df_men.shape[0] == df_women.shape[0]

True

### В мужском населениии за 2021 год есть пропуски. Их надо восстановить
1) Через соотношение численности мужчин и женщин, предполагая, что оно не сильно меняется
2) Линенейная регрессия

In [133]:
df_women_1 = df_women[df_women['age_category'] == 1].reset_index(drop = True)
df_men_1 = df_men[df_men['age_category'] == 1].reset_index(drop = True)

df_rel = df_men_1.select_dtypes(include = ['object']).copy()
year_cols = [col for col in df_men.columns if "end" in col]

for col in year_cols:
    df_rel[col] = df_women_1[col] / df_men_1[col]
df_rel['age_category'] = df_men_1['age_category']

In [115]:
df_rel.to_excel("ratio.xlsx", index = False)

In [136]:
mean_2021 = (df_rel['2020end'] + df_rel['2022end']) / 2
df_rel["2021end"] = df_rel["2021end"].fillna(mean_2021)

In [153]:
men_2021 = df_men_1["2021end"] * df_rel["2021end"]
men_2021 = men_2021.replace([np.inf, -np.inf], np.nan)
df_men_1["2021end"] = df_men_1["2021end"].astype("float")
df_men_1["2021end"] = df_men_1["2021end"].fillna(men_2021)

In [154]:
pd.set_option("display.max_columns", None)
df_men_1.head()

,region,age,settlement,1990end,1991mid,1991end,1992mid,1992end,1993mid,1993end,1994mid,1994end,1995mid,1995end,1996mid,1996end,1997mid,1997end,1998mid,1998end,1999mid,1999end,2000mid,2000end,2001mid,2001end,2002mid,2002end,2003mid,2003end,2004mid,2004end,2005mid,2005end,2006mid,2006end,2007mid,2007end,2008mid,2008end,2009mid,2009end,2010mid,2010end,2011mid,2011end,2012mid,2012end,2013mid,2013end,2014mid,2014end,2015mid,2015end,2016mid,2016end,2017mid,2017end,2018mid,2018end,2019mid,2019end,2020mid,2020end,2021mid,2021end,2022mid,2022end,2023mid,2023end,2024mid,2024end,2025mid,2025end,min_age,max_age,age_category
0,Российская Федерация,0,все население,1087571,1047535.0,1007499,959429.0,911359,862700.5,814042,763712.0,713382,724793.0,736204,719871.5,703539,689218.5,674898,662641.5,650385,656117.5,661850,644816.0,627782,639515.0,651248,657313.5,663379,683304.5,703230,727264.5,751299,756893.0,762487,752798.0,743109,747225.0,751341,785603.5,819866,844951.0,870036,878167.0,886298,869388.0,852478,883036.0,913594,941094.0,968594,965560.0,962526,974999.5,987473,991783.0,996093,980220.5,964348,914095.5,863843,842822.0,821801,789787.5,757774,<NA>,<NA>,<NA>,714446,691336.0,668226,659125.5,650025,640359.5,630694,0.0,0.0,1
1,Российская Федерация,0,городское население,767215,735244.0,703273,664478.0,625683,585280.0,544877,510583.5,476290,486742.0,497194,487296.5,477399,469013.0,460627,453018.0,445409,449484.0,453559,442172.0,430785,441087.5,451390,457191.5,462993,479496.0,495999,515433.0,534867,538307.0,541747,534883.0,528019,529552.5,531086,551445.0,571804,588829.0,605854,613685.5,621517,612963.5,604410,624612.5,644815,667616.0,690417,689533.0,688649,699323.5,709998,726018.0,742038,733734.0,725430,686430.5,647431,631875.5,616320,592890.0,569460,<NA>,<NA>,<NA>,537022,519595.0,502168,495496.5,488825,483194.5,477564,0.0,0.0,1
2,Российская Федерация,0,сельское население,320356,312291.0,304226,294951.0,285676,277420.5,269165,253128.5,237092,238051.0,239010,232575.0,226140,220205.5,214271,209623.5,204976,206633.5,208291,202644.0,196997,198427.5,199858,200122.0,200386,203808.5,207231,211831.5,216432,218586.0,220740,217915.0,215090,217672.5,220255,234158.5,248062,256122.0,264182,264481.5,264781,256424.5,248068,258423.5,268779,273478.0,278177,276027.0,273877,275676.0,277475,265765.0,254055,246486.5,238918,227665.0,216412,210946.5,205481,196897.5,188314,<NA>,<NA>,<NA>,177424,171741.0,166058,163629.0,161200,157165.0,153130,0.0,0.0,1
3,Российская Федерация,1,все население,1178636,1130834.5,1083033,1046561.0,1010089,961233.5,912378,864359.0,816340,767993.0,719646,727797.5,735949,719760.0,703571,688723.5,673876,661330.5,648785,654482.0,660179,644372.0,628565,638528.0,648491,651759.0,655027,680658.5,706290,726952.0,747614,752211.5,756809,749755.5,742702,745704.0,748706,782394.0,816082,839854.5,863627,862562.0,861497,858251.5,855006,884034.0,913062,940330.0,967598,971953.5,976309,981917.5,987526,991600.0,995674,979972.5,964271,914081.5,863892,843042.5,822193,<NA>,<NA>,<NA>,731314,723334.5,715355,692132.5,668910,660203.0,651496,1.0,1.0,1
4,Российская Федерация,1,городское население,833156,798796.5,764437,733271.5,702106,661209.0,620312,582792.0,545272,513086.5,480901,489286.5,497672,487690.5,477709,468743.0,459777,451848.5,443920,447245.0,450570,440620.5,430671,438918.5,447166,451691.5,456217,477639.5,499062,513642.0,528222,531935.5,535649,531583.5,527518,528362.5,529207,548772.0,568337,584596.5,600856,604285.0,607714,607111.5,606509,625709.5,644910,667692.0,690474,693845.0,697216,704150.5,711085,727073.0,743061,734921.0,726781,687832.0,648883,633360.0,617837,<NA>,<NA>,<NA>,551322,544867.5,538413,520807.5,503202,497014.5,490827,1.0,1.0,1
